<a href="https://colab.research.google.com/github/nithinrk11/Feynn-Labs-Machine-Learning-Internship/blob/main/Project%203/diab_reci_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
!git clone https://github.com/nithinrk11/Feynn-Labs-Machine-Learning-Internship.git
%cd Feynn-Labs-Machine-Learning-Internship

Cloning into 'Feynn-Labs-Machine-Learning-Internship'...
remote: Enumerating objects: 402, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 402 (delta 156), reused 163 (delta 89), pack-reused 127 (from 1)
Receiving objects: 100% (402/402), 16.05 MiB | 15.49 MiB/s, done.
Resolving deltas: 100% (196/196), done.
/content/Feynn-Labs-Machine-Learning-Internship/Feynn-Labs-Machine-Learning-Internship/Feynn-Labs-Machine-Learning-Internship/Feynn-Labs-Machine-Learning-Internship/Feynn-Labs-Machine-Learning-Internship


##Installing dependencies

In [62]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle

In [63]:
data = pd.read_csv('/content/Feynn-Labs-Machine-Learning-Internship/Project 3/filtered_datasets/merged_recipe_df.csv')

In [64]:
data.head()

,Unnamed: 0,Recipe Name,Ingredients,Calories,Sugar (g),Total Fat (g),Saturated Fat (g),Cuisine
0,0,Chicken with Curry Leaves from Andhra Pradesh ...,"3 medium chicken breasts, cut into bite-sized ...",2002.334204,51.733198,73.758990,32.176384,Andhra Pradesh
1,1,Punugulu,"2 Cup Idli/Dosa batter, 0.5 Cup Besan flour, 0...",1057.899202,13.100185,3.945037,0.767981,Andhra Pradesh
2,2,Bid goodbye to grey boiled eggs thanks to this...,"1/3 cup (80ml) sunflower oil, plus extra 1/3 c...",1974.410794,50.171840,133.102144,26.887364,Andhra Pradesh
3,3,Andhra Gunpowder,1 cup roasted gram (also known as chana dalia ...,134.521833,4.477519,5.776929,3.226080,Andhra Pradesh
4,4,Ginger Chutney,"1/2 cup peeled, grated ginger, 1 1/4 teaspoons...",75.018000,7.984223,0.785400,0.163840,Andhra Pradesh


In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641 entries, 0 to 640
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         641 non-null    int64  
 1   Recipe Name        641 non-null    object 
 2   Ingredients        641 non-null    object 
 3   Calories           641 non-null    float64
 4   Sugar (g)          641 non-null    float64
 5   Total Fat (g)      641 non-null    float64
 6   Saturated Fat (g)  641 non-null    float64
 7   Cuisine            641 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 40.2+ KB


* Setting the sugar level criteria for diabetic types
> although this is set in an average estimated values, by referring to various medical guidlines sources, setting an proper prescribed levels by an professional is preferred.

In [66]:
# Define the diabetic-friendly criteria based on sugar, calories, and fat
def is_diabetic_friendly(sugar, calories, total_fat):
    return (sugar < 5) and (calories < 500) and (total_fat < 20)

def is_non_diabetic_friendly(sugar, calories, total_fat):
    return (sugar < 10) and (calories < 800) and (total_fat < 30)

# Apply criteria to create new columns
data['diabetic_friendly'] = data.apply(lambda row: is_diabetic_friendly(row['Sugar (g)'], row['Calories'], row['Total Fat (g)']), axis=1)
data['non_diabetic_friendly'] = data.apply(lambda row: is_non_diabetic_friendly(row['Sugar (g)'], row['Calories'], row['Total Fat (g)']), axis=1)


In [67]:
data.head()

,Unnamed: 0,Recipe Name,Ingredients,Calories,Sugar (g),Total Fat (g),Saturated Fat (g),Cuisine,diabetic_friendly,non_diabetic_friendly
0,0,Chicken with Curry Leaves from Andhra Pradesh ...,"3 medium chicken breasts, cut into bite-sized ...",2002.334204,51.733198,73.758990,32.176384,Andhra Pradesh,False,False
1,1,Punugulu,"2 Cup Idli/Dosa batter, 0.5 Cup Besan flour, 0...",1057.899202,13.100185,3.945037,0.767981,Andhra Pradesh,False,False
2,2,Bid goodbye to grey boiled eggs thanks to this...,"1/3 cup (80ml) sunflower oil, plus extra 1/3 c...",1974.410794,50.171840,133.102144,26.887364,Andhra Pradesh,False,False
3,3,Andhra Gunpowder,1 cup roasted gram (also known as chana dalia ...,134.521833,4.477519,5.776929,3.226080,Andhra Pradesh,True,True
4,4,Ginger Chutney,"1/2 cup peeled, grated ginger, 1 1/4 teaspoons...",75.018000,7.984223,0.785400,0.163840,Andhra Pradesh,False,True


In [68]:
data.columns

Index(['Unnamed: 0', 'Recipe Name', 'Ingredients', 'Calories', 'Sugar (g)',
       'Total Fat (g)', 'Saturated Fat (g)', 'Cuisine', 'diabetic_friendly',
       'non_diabetic_friendly'],
      dtype='object')

##Save and download the new dataframe

In [69]:
#save the new dataframe
data.to_csv('diab_merged_recipe_df.csv', index=False)


In [70]:
#download the csv file
from google.colab import files
files.download('diab_merged_recipe_df.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Machine learning

In [71]:
# Features and targets
X = data[['Calories', 'Sugar (g)', 'Total Fat (g)', 'Saturated Fat (g)']]
y = data[['diabetic_friendly', 'non_diabetic_friendly']]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model for each obesity type
models = {}
for column in y.columns:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train[column])
    models[column] = model

# Evaluate the model
for column, model in models.items():
    y_pred = model.predict(X_test)
    print(f"Classification Report for {column}:\n", classification_report(y_test[column], y_pred))

Classification Report for diabetic_friendly:
               precision    recall  f1-score   support

       False       1.00      1.00      1.00       127
        True       1.00      1.00      1.00         2

    accuracy                           1.00       129
   macro avg       1.00      1.00      1.00       129
weighted avg       1.00      1.00      1.00       129

Classification Report for non_diabetic_friendly:
               precision    recall  f1-score   support

       False       1.00      1.00      1.00       123
        True       1.00      1.00      1.00         6

    accuracy                           1.00       129
   macro avg       1.00      1.00      1.00       129
weighted avg       1.00      1.00      1.00       129



##Save the trained model

In [72]:
# Save the trained models to a pickle file
with open('trained_diab_recep_model.pkl', 'wb') as file:
    pickle.dump(models, file)


In [73]:
from google.colab import files

files.download('trained_diab_recep_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Make predictions based on diabetic type

In [74]:
#load the trained model and predict diabetic_friendly is true to predict food nameand thier cusine, sugar, calorie, fat levels from the dataset in a dataframe

import pandas as pd
import pickle

# Load the trained model
with open('trained_diab_recep_model.pkl', 'rb') as file:
    loaded_models = pickle.load(file)

# Get the model for 'diabetic_friendly'
diabetic_model = loaded_models['diabetic_friendly']

# Make predictions for 'diabetic_friendly'
X_new = data[['Calories', 'Sugar (g)', 'Total Fat (g)', 'Saturated Fat (g)']]
diabetic_predictions = diabetic_model.predict(X_new)

# Filter the DataFrame for recipes predicted as diabetic_friendly
diabetic_friendly_recipes = data[diabetic_predictions == True]

# Select the desired columns (Food name, Cuisine, Sugar, Calories, Fat)
result_df = diabetic_friendly_recipes[['Recipe Name', 'Cuisine', 'Sugar (g)', 'Calories', 'Total Fat (g)']]

# Print the resulting DataFrame
result_df



,Recipe Name,Cuisine,Sugar (g),Calories,Total Fat (g)
3,Andhra Gunpowder,Andhra Pradesh,4.477519,134.521833,5.776929
185,Hyderabadi Lamb Biryani,Hyderabadi,0.071394,19.324959,0.705807
229,"Prawns Curry, Kerala Style",Kerala,0.393990,369.329583,5.687433
270,Kerala Style Spicy Tangy Fish Curry,Kerala,0.227485,13.593500,0.457745
272,Garam Masala--- Kerala Style,Kerala,0.910630,275.141500,13.370450
282,kerala style red coconut chutney recipe,Kerala,0.714817,144.099000,13.975973
296,Kerala Style Parippu Curry for Sadya ~ Len,Kerala,4.360900,168.872000,14.083200
331,Varan ~ Maharashtrian Dal,Maharashtrian,1.758830,208.446873,15.404634
357,Kothmir Wadi | Kothambir Wadi – Maharashtrian ...,Maharashtrian,0.883457,454.713771,16.682675
429,punjabi mooli paratha recipe,Punjabi,3.758943,31.974750,0.173239
